Lesson Overview:

In this lesson, we are going to discuss types of ANNs and Recurrent Neural Network.

Use Case Description:

1. Sentiment Analysis on the Twitter dataset
Programming elements:
1. Basics of LSTM
2. Types of RNN
3. Use case: Sentiment Analysis on the Twitter data set

In [1]:
import pandas as pd  # Importing pandas for data processing
from keras.preprocessing.text import Tokenizer  # Importing Tokenizer for text preprocessing
from keras.utils import pad_sequences  # Importing pad_sequences for sequence padding
from keras.models import Sequential  # Importing Sequential for building a sequential model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D  # Importing various layers for the LSTM model
from matplotlib import pyplot  # Importing matplotlib for plotting
from sklearn.model_selection import train_test_split , GridSearchCV  # Importing train_test_split for data splitting and grid search cv
from keras.utils.np_utils import to_categorical  # Importing to_categorical for one-hot encoding
import re  # Importing re for regular expression operations
from sklearn.preprocessing import LabelEncoder  # Importing LabelEncoder for label encoding
import numpy as np # Importing Numpy
from keras.wrappers.scikit_learn import KerasClassifier  # Importing kerasclassfier

In [2]:
# Read the data from 'Sentiment.csv' and keep only the 'text' and 'sentiment' columns
data = pd.read_csv('Sentiment.csv')
data = data[['text', 'sentiment']]

In [3]:
# Convert text to lowercase and remove special characters using regular expressions
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [4]:
# Replace 'rt' (retweet) with a space in the text
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [5]:
max_fatures = 2000  # Set the maximum number of unique words to 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')  # Create a tokenizer with the specified max_fatures
tokenizer.fit_on_texts(data['text'].values)  # Fit the tokenizer on the text data
X = tokenizer.texts_to_sequences(data['text'].values)  # Convert text to sequences using the tokenizer

In [6]:
X = pad_sequences(X)  # Pad the sequences to make them of equal length

In [7]:
embed_dim = 128  # Set the embedding dimension to 128
lstm_out = 196  # Set the LSTM output dimension to 196

In [8]:
# Function to create the LSTM model
def createmodel():
    model = Sequential()  # Create a sequential model
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))  # Add an Embedding layer
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))  # Add an LSTM layer with dropout
    model.add(Dense(3, activation='softmax'))  # Add a Dense output layer with softmax activation
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Compile the model
    return model

In [9]:
# Encode the 'sentiment' labels using LabelEncoder and convert them into one-hot encoded vectors
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

In [10]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 32  # Set the batch size to 32
model = createmodel()  # Create the LSTM model using the createmodel function
model.fit(X_train, Y_train, epochs=7, batch_size=batch_size, verbose=2)  # Train the model for seven epoch
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)  # Evaluate the model on the test set

# Print the loss, accuracy, and metric names
print(score)
print(acc)
print(model.metrics_names)

Epoch 1/7
291/291 - 61s - loss: 0.8265 - accuracy: 0.6482 - 61s/epoch - 208ms/step
Epoch 2/7
291/291 - 41s - loss: 0.6794 - accuracy: 0.7114 - 41s/epoch - 140ms/step
Epoch 3/7
291/291 - 39s - loss: 0.6121 - accuracy: 0.7471 - 39s/epoch - 134ms/step
Epoch 4/7
291/291 - 38s - loss: 0.5711 - accuracy: 0.7651 - 38s/epoch - 131ms/step
Epoch 5/7
291/291 - 38s - loss: 0.5233 - accuracy: 0.7839 - 38s/epoch - 131ms/step
Epoch 6/7
291/291 - 38s - loss: 0.4842 - accuracy: 0.8049 - 38s/epoch - 130ms/step
Epoch 7/7
291/291 - 37s - loss: 0.4427 - accuracy: 0.8206 - 37s/epoch - 129ms/step
144/144 - 2s - loss: 0.9148 - accuracy: 0.6603 - 2s/epoch - 11ms/step
0.9147756099700928
0.6603320240974426
['loss', 'accuracy']


In class programming:
1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are
happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [11]:
# After training the model (model.fit), save the model using model.save()
model.save('sentiment_analysis_model.h5')

# Later, load the model using keras.models.load_model()
from keras.models import load_model
loaded_model = load_model('sentiment_analysis_model.h5')

# Now, you can use the loaded model to predict on new text data
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"
new_text = new_text.lower()
new_text = re.sub('[^a-zA-z0-9\s]', '', new_text)
new_text_sequence = tokenizer.texts_to_sequences([new_text])
new_text_padded = pad_sequences(new_text_sequence, maxlen=X.shape[1])

# Predict the sentiment for the new text
predictions = loaded_model.predict(new_text_padded)
# Decoding the predictions as we have encoded the the sentiments
sentiment_labels = ['Negative', 'Neutral', 'Positive']
predicted_sentiment = sentiment_labels[np.argmax(predictions)]
print("Predicted Sentiment:", predicted_sentiment)


1/1 [==============================] - 0s 497ms/step
Predicted Sentiment: Positive


2. Apply GridSearchCV on the source code provided in the class

In [ ]:
# Function to create the LSTM model
def createmodel(optimizer, weight_initializer, activation, dropout_rate):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(3, activation=activation))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Wrap the Keras model using KerasClassifier
model = KerasClassifier(build_fn=createmodel, verbose=0)

# Define the parameter grid to search over
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'weight_initializer': ['uniform', 'glorot_uniform'],
    'activation': ['relu', 'sigmoid'],
    'dropout_rate': [0.2, 0.3]
}

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, Y_train)


In [19]:

# Print the best parameters and accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Predict on the test set using the best model
best_model = grid_result.best_estimator_
acc = best_model.score(X_test, Y_test)
print("Test Accuracy: %.2f%%" % (acc * 100))

Best: 0.669967 using {'activation': 'sigmoid', 'dropout_rate': 0.2, 'optimizer': 'adam', 'weight_initializer': 'uniform'}
Test Accuracy: 67.26%
